0) Import necessary packages/libraries and create useful functions for convenient use 

In [1]:
"""
Created on Wed Apr 22 15:28:32 2020

@author: Pranav Ashok
"""

# Import and check the versions of packages
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))

import pandas as pd
print('pandas: {}'.format(pd.__version__))

import os
from IPython.display import display, HTML

#Navigate to directory with the country data
originalDir = os.getcwd()
os.chdir(originalDir + '/usa_data/csv')
thisDir = os.getcwd()

matplotlib: 3.1.3
pandas: 1.0.1


In [2]:
# Handle date changes when looping through data
def updateDate(year, month, date):
    newYear = year
    newMonth = month
    newDate = date
    if date == 30 and (month==4 or month==6 or month==9 or month==11):
      newMonth+=1
      newDate = 1

    elif (month==2 and date==28 and year%4!=0) or (month==2 and date==29 and year%4==0):
      newMonth+=1
      newDate = 1

    elif date==31:
        if month==12:
            newMonth=1
            newYear+=1
        else:
            newMonth+=1
        newDate = 1

    else:
        newDate+=1

    newDate = [newYear, newMonth, newDate]
    return newDate

In [3]:
# Export dataframe as CSV file
def dfToCSV(df, nameOfCSV):
    df.to_csv(nameOfCSV + '.csv')

In [4]:
# Function to update dataset with new data
def updateDataset(startYear, startMonth, startDate, endYear, endMonth, endDate, df, features):
    year = startYear
    month = startMonth
    date = startDate
    index = 1

    firstData = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
    newDF = pd.DataFrame(data=firstData)
    newDF = newDF.loc[(newDF[newDF.columns.values[0]] == 'United States') | (newDF[newDF.columns.values[0]] == 'USA')]

    while year!=endYear or month!=endMonth or date!=endDate:
        lastCSVdata = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
        lastDF = pd.DataFrame(data=lastCSVdata)

        newDate = updateDate(year, month, date)
        year = newDate[0]
        month = newDate[1]
        date = newDate[2]

        CSVdata = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
        tempDF = pd.DataFrame(data=CSVdata)
        tempDF = tempDF.loc[(tempDF[tempDF.columns.values[0]] == 'United States') | (tempDF[tempDF.columns.values[0]] == 'USA') | (tempDF[tempDF.columns.values[0]] == 'U.S.')]  
        if(tempDF.empty or len(tempDF)==0 or tempDF.equals(lastDF)):
            print('Error loading data from: ' + str(month) + '-' + str(date) + '-' + str(year))
        newDF = newDF.append(tempDF, sort=False)
        index+=1
    newDF = newDF[features]
    date_index = pd.date_range(str(startMonth) + '/' + str(startDate) + '/' + str(startYear), periods=index, freq='D')
    newDF = newDF.set_index(date_index)
    return newDF

In [5]:
# Function used for feature scaling using standardization (z-scores)
def scaleFeatures(df):
    means = df.mean().tolist()    
    stdDevs = df.std().tolist()
        
    rows = range(len(df))
    columns = range(len(df.columns))
    
    for colNum in columns:
        for rowNum in rows:
            df.iat[rowNum, colNum] = (df.iat[rowNum, colNum] - means[colNum])/stdDevs[colNum]

1) Combine CSV data for USA

In [6]:
year = 2020
month = 1
date = 29

currentYear = 2020
currentMonth = 4
currentDate = 24
firstData = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
USdf = pd.DataFrame(data=firstData)
USdf = USdf.loc[(USdf[USdf.columns.values[0]] == 'United States') | (USdf[USdf.columns.values[0]] == 'USA')]

while year!=currentYear or month!=currentMonth or date!=currentDate:
    lastCSVdata = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
    lastDF = pd.DataFrame(data=lastCSVdata)
    
    newDate = updateDate(year, month, date)
    year = newDate[0]
    month = newDate[1]
    date = newDate[2]
    
    CSVdata = pd.read_csv(str(month) + '_' + str(date) + '_' + str(year) + '.csv')
    tempDF = pd.DataFrame(data=CSVdata)
    tempDF = tempDF.loc[(tempDF[tempDF.columns.values[0]] == 'United States') | (tempDF[tempDF.columns.values[0]] == 'USA') | (tempDF[tempDF.columns.values[0]] == 'U.S.')]  
    if(tempDF.empty or len(tempDF)==0 or tempDF.equals(lastDF)):
        print('Error loading data from: ' + str(month) + '-' + str(date) + '-' + str(year))
    USdf = USdf.append(tempDF, sort=False)

2) Clean up data

In [7]:
date_index = pd.date_range('1/30/2020', periods=87, freq='D')
USdf = USdf.set_index(date_index)

drops = [USdf.columns[0], USdf.columns[2], USdf.columns[3], USdf.columns[5], USdf.columns[7], USdf.columns[6], USdf.columns[9], USdf.columns[10], USdf.columns[12], USdf.columns[13], USdf.columns[15], USdf.columns[16], USdf.columns[17], USdf.columns[18], USdf.columns[19], USdf.columns[20], USdf.columns[21], USdf.columns[22], USdf.columns[23], USdf.columns[25], USdf.columns[26], USdf.columns[27], USdf.columns[28], USdf.columns[29], USdf.columns[30], USdf.columns[31], USdf.columns[32], USdf.columns[33], USdf.columns[34], USdf.columns[36], USdf.columns[37], USdf.columns[38], USdf.columns[41], USdf.columns[42], USdf.columns[43], USdf.columns[44], USdf.columns[45], USdf.columns[47], USdf.columns[48]]
USdf = USdf.drop(columns=drops)

USdf = USdf.iloc[30:, :]

3) Handle missing data

In [8]:
USdf['Total\nCases'] = USdf['Total\nCases'].fillna(USdf['Cases'])
USdf['Total\nCases'] = USdf['Total\nCases'].fillna(USdf['Total Cases'])
USdf = USdf.iloc[:, [2, 4, 6, 7, 8, 9]]

USdf['Total\nDeaths'] = USdf['Total\nDeaths'].fillna(value=0)
USdf['Total\nRecovered'] = USdf['Total\nRecovered'].fillna(value=0)
USdf[USdf.columns[2]] = USdf[USdf.columns[2]].fillna(value=0)

testData = pd.read_csv('test_data.csv')
testDF = pd.DataFrame(data=testData)
testDF = testDF.loc[testDF[testDF.columns.values[0]] == 'United States']
totalTests = testDF[testDF.columns[3]]
totalTests = [0, 0, 0, 0, 0, 0, 0, 0] + totalTests.tolist() + [4940376]
USdf.insert(5, "Total Tests", totalTests, True)
USdf = USdf.iloc[:, 0:6]
USdf = USdf[[USdf.columns[1], USdf.columns[2], USdf.columns[3], USdf.columns[4], USdf.columns[5], USdf.columns[0]]]

4) Update dataset with new incoming data

In [9]:
# Use update function to add new data to dataframe
features = [USdf.columns[0], USdf.columns[1], USdf.columns[2], USdf.columns[3], 'Total\nTests', USdf.columns[5]]
newDF = updateDataset(2020, 4, 25, 2020, 4, 28, USdf, features)
USdf = USdf.append(newDF, sort=False)

USdf['Total\nTests'] = USdf['Total\nTests'].fillna(USdf['Total Tests'])
USdf = USdf[[USdf.columns[0], USdf.columns[1], USdf.columns[2], USdf.columns[3], USdf.columns[6], USdf.columns[5]]]

dfToCSV(USdf, 'USA_dataset_raw')

,Total Recovered,"Serious, Critical",Total Cases,Active Cases,Total Tests,Total Deaths
2020-02-29,6.0,1.0,63.0,57.0,0.0,0.0
2020-03-01,7.0,1.0,68.0,60.0,0.0,1.0
2020-03-02,9.0,4.0,75.0,65.0,0.0,1.0
2020-03-03,9.0,7.0,99.0,84.0,0.0,6.0
2020-03-04,9.0,7.0,118.0,100.0,0.0,9.0
2020-03-05,9.0,8.0,158.0,138.0,0.0,11.0
2020-03-06,9.0,8.0,221.0,200.0,0.0,12.0
2020-03-07,15.0,8.0,316.0,286.0,0.0,15.0
2020-03-08,15.0,8.0,434.0,400.0,2390.0,19.0
2020-03-09,15.0,8.0,538.0,501.0,3069.0,22.0


5) Scale features so that gradient descent will converge (and faster)

In [10]:
#Move deaths data one day forward for model to predict the next days' deaths
tmrwTotalDeaths = USdf[USdf.columns[5]].tolist()
tmrwTotalDeaths = tmrwTotalDeaths + [59266.0]
tmrwTotalDeaths.pop(0)
USdf.insert(6, "Total\nDeaths\nTomorrow", tmrwTotalDeaths)
USdf = USdf.drop(columns=[USdf.columns[5]])
display(HTML(USdf.to_html()))

# Feature scaling
scaleFeatures(USdf)

,Total Recovered,"Serious, Critical",Total Cases,Active Cases,Total Tests,Total Deaths Tomorrow
2020-02-29,6.0,1.0,63.0,57.0,0.0,1.0
2020-03-01,7.0,1.0,68.0,60.0,0.0,1.0
2020-03-02,9.0,4.0,75.0,65.0,0.0,6.0
2020-03-03,9.0,7.0,99.0,84.0,0.0,9.0
2020-03-04,9.0,7.0,118.0,100.0,0.0,11.0
2020-03-05,9.0,8.0,158.0,138.0,0.0,12.0
2020-03-06,9.0,8.0,221.0,200.0,0.0,15.0
2020-03-07,15.0,8.0,316.0,286.0,0.0,19.0
2020-03-08,15.0,8.0,434.0,400.0,2390.0,22.0
2020-03-09,15.0,8.0,538.0,501.0,3069.0,26.0


6) Export final dataset as CSV file

In [11]:
dfToCSV(USdf, 'USA_dataset')